In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Import base classifiers
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from baselines import AdaFairClassifier
from imbens.ensemble import SMOTEBoostClassifier, SMOTEBaggingClassifier, RUSBoostClassifier, UnderBaggingClassifier, SelfPacedEnsembleClassifier
from fairlearn.postprocessing import ThresholdOptimizer
from fairens import FairAugEnsemble, FairEnsemble

# Import utilities
from data import FairDataset    # This is a custom class that we will use to load the datasets
from eval import evaluate_multi_split, verbose_print
from trainer import Trainer
from utils import seed_generator, dict_info

pip install 'aif360[AdversarialDebiasing]'


In [2]:
"""Load Datasets"""

dataset_kwargs = {
    'y_col': 'label',
    'train_size': 0.6,
    'val_size': 0.2,
    'test_size': 0.2,
    'concat_train_val': True,
    'normalize': True,
    'random_state': 42,
}

all_datasets = {
    'compas': ['sex', 'race'],
    'adult': ['gender', 'race'],
    'lsa_unfair_gender_race': ['gender', 'race'],
    'bank': ['age', 'marital=married'],
}

"""
Create a dictionary of datasets: dataset_zoo
key: dataset name
value: FairDataset object
"""
dataset_zoo = {}
for dataname, s_attrs in all_datasets.items():
    for s_attr in s_attrs:
        dataset = FairDataset(
            dataname=dataname,
            csv_path=f'./data/{dataname}.csv',
            s_col=s_attr,
            **dataset_kwargs
        )
        dataset_zoo[dataset.fullname] = dataset

        # dataset.describe()
        dataset.brief()

# Print the information of the datasets and models
print(
    f"////// Dataset ZOO //////\n"
    f"{dict_info(dataset_zoo)}\n"
)

Dataset    : compas (5875, 12) load from ./data/compas.csv
Sens/Res   : sex/label
Split      : train/test = 0.8/0.2, random_state = 42, x_with_s = True
train      | size {0: 929, 1: 3771} | grp_pos_ratio: {0: 0.3617, 1: 0.4916}
test       | size {0: 232, 1: 943} | grp_pos_ratio: {0: 0.3621, 1: 0.491}

Dataset    : compas (5875, 12) load from ./data/compas.csv
Sens/Res   : race/label
Split      : train/test = 0.8/0.2, random_state = 42, x_with_s = True
train      | size {0: 1878, 1: 2822} | grp_pos_ratio: {0: 0.3946, 1: 0.5135}
test       | size {0: 469, 1: 706} | grp_pos_ratio: {0: 0.3945, 1: 0.5127}

Dataset    : adult (45222, 99) load from ./data/adult.csv
Sens/Res   : gender/label
Split      : train/test = 0.8/0.2, random_state = 42, x_with_s = True
train      | size {0: 11756, 1: 24421} | grp_pos_ratio: {0: 0.1136, 1: 0.3125}
test       | size {0: 2939, 1: 6106} | grp_pos_ratio: {0: 0.1136, 1: 0.3125}

Dataset    : adult (45222, 99) load from ./data/adult.csv
Sens/Res   : race/labe

In [3]:
from trainer import Benchmarker
from baselines import ReweightClassifier, ReductionClassifier

ensemble_kwargs = {
    'n_estimators': 5,
    'random_state': 42,
}
single_ensemble_kwargs = {
    'n_estimators': 1,
    'random_state': 42,
}

base_models = {
    'LR': LogisticRegression(),
    'KN': KNeighborsClassifier(),
    'DT': DecisionTreeClassifier(max_depth=None),
    'MLP': MLPClassifier(hidden_layer_sizes=(8), max_iter=50),
    'ADA': AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=None), n_estimators=5),
    'BAG': BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=None), n_estimators=5),
}

baselines = {
    # 'AdaBoost': (AdaBoostClassifier, {**ensemble_kwargs}),
    # 'Bagging': (BaggingClassifier, {**ensemble_kwargs}),
    # 'RUSBoost': (RUSBoostClassifier, {**ensemble_kwargs}),
    # 'UnderBag': (UnderBaggingClassifier, {**ensemble_kwargs}),
    # 'SMBoost': (SMOTEBoostClassifier, {**ensemble_kwargs}),
    # 'SMBag': (SMOTEBaggingClassifier, {**ensemble_kwargs}),
    'Reweight': (ReweightClassifier, {}),
    'ReductionDP': (ReductionClassifier, {'constraints': 'DemographicParity'}),
    'ReductionEO': (ReductionClassifier, {'constraints': 'EqualizedOdds'}),
    'ThresDP': (ThresholdOptimizer, {'constraints': 'demographic_parity'}),
    'ThresEO': (ThresholdOptimizer, {'constraints': 'equalized_odds'}),
    'AdaFair': (AdaFairClassifier, {'saIndex': 0, 'saValue': 0, 'CSB': 'CSB2', **ensemble_kwargs}),
    # 'Ours-SubOpt': (FairAugEnsemble, {
    #     'how': {'mode': 'sub_optim'}, **ensemble_kwargs
    # }),
    'Ours': (FairEnsemble, {
        'how': {'pos_ratio': 'max', 'unifomity': 0.1, 'drop_ratio': 1, 'bootstrap': False}, **ensemble_kwargs
    }),
    'Ours-single': (FairEnsemble, {
        'how': {'pos_ratio': 'max', 'uniformity': 0.1, 'drop_ratio': 1, 'bootstrap': False}, **single_ensemble_kwargs
    }),
    'Ours-uniform-0.01': (FairEnsemble, {
        'how': {'pos_ratio': 'max', 'uniformity': 0.01, 'drop_ratio': 1, 'bootstrap': False}, **ensemble_kwargs
    }),
    'Ours-uniform-0.01-single': (FairEnsemble, {
        'how': {'pos_ratio': 'max', 'uniformity': 0.01, 'drop_ratio': 1, 'bootstrap': False}, **single_ensemble_kwargs
    }),
    'Ours-uniform-1.0': (FairEnsemble, {
        'how': {'pos_ratio': 'max', 'uniformity': 1, 'drop_ratio': 1, 'bootstrap': False}, **ensemble_kwargs
    }),
}

benchmark = Benchmarker(
    base_models=base_models,
    baselines=baselines,
    datasets=dataset_zoo,
    random_state=42,
    dummy_strategy='stratified',
)
benchmark.run(n_runs=5, group_by='dataset', exception='ignore')

Initializing Benchmarker with:
Random seed: 42
Base models: ['LR', 'KN', 'DT', 'MLP', 'ADA', 'BAG']
Techniques:  ['Reweight', 'ReductionDP', 'ReductionEO', 'ThresDP', 'ThresEO', 'AdaFair', 'Ours', 'Ours-single', 'Ours-uniform-0.01', 'Ours-uniform-0.01-single', 'Ours-uniform-1.0']
Datasets:    ['compas_sex', 'compas_race', 'adult_gender', 'adult_race', 'lsa_unfair_gender_race_gender', 'lsa_unfair_gender_race_race', 'bank_age', 'bank_marital=married']
# models:    72
# datasets:  8

Running All models ...
========== Start Running on Dataset: compas_sex ==========


Data: compas_sex | Model: Dummy                        : 100%|██████████| 5/5 [00:00<00:00, 17.79it/s, ACC 0.497±0.021 | BACC 0.495±0.022 | DP 0.010±0.006 | EO 0.044±0.043 | AdvG {0: 4, 1: 1}]
Data: compas_sex | Model: LR                           : 100%|██████████| 5/5 [00:00<00:00,  6.45it/s, ACC 0.696±0.006 | BACC 0.690±0.006 | DP 0.336±0.016 | EO 0.382±0.039 | AdvG {1: 5}]
Data: compas_sex | Model: LR_Reweight                  : 100%|██████████| 5/5 [00:03<00:00,  1.58it/s, ACC 0.689±0.009 | BACC 0.677±0.009 | DP 0.275±0.022 | EO 0.343±0.055 | AdvG {1: 5}]
Data: compas_sex | Model: LR_ReductionDP               : 100%|██████████| 5/5 [00:07<00:00,  1.41s/it, ACC 0.689±0.010 | BACC 0.686±0.010 | DP 0.105±0.041 | EO 0.086±0.043 | AdvG {1: 5}]
Data: compas_sex | Model: LR_ReductionEO               : 100%|██████████| 5/5 [00:07<00:00,  1.55s/it, ACC 0.687±0.009 | BACC 0.682±0.009 | DP 0.128±0.028 | EO 0.109±0.033 | AdvG {1: 5}]
Data: compas_sex | Model: LR_ThresDP                   : 10

========== Run Time on Dataset compas_sex: 2m16s ==========
Results on Data: compas_sex
Model: Dummy                        | ACC 0.497±0.021            | BACC 0.495±0.022            | DP 0.010±0.006            | EO 0.044±0.043           
Model: LR                           | ACC 0.696±0.006            | BACC 0.690±0.006            | DP 0.336±0.016            | EO 0.382±0.039           
Model: LR_Reweight                  | ACC 0.689±0.009 (-3.59%)   | BACC 0.677±0.009 (-6.74%)   | DP 0.275±0.022 (-18.18%)  | EO 0.343±0.055 (-10.36%)  | FURG 9.10     | FUTR 2.76    
Model: LR_ReductionDP               | ACC 0.689±0.010 (-3.42%)   | BACC 0.686±0.010 (-2.08%)   | DP 0.105±0.041 (-68.82%)  | EO 0.086±0.043 (-77.60%)  | FURG 70.46    | FUTR 26.62   
Model: LR_ReductionEO               | ACC 0.687±0.009 (-4.36%)   | BACC 0.682±0.009 (-3.93%)   | DP 0.128±0.028 (-61.80%)  | EO 0.109±0.033 (-71.38%)  | FURG 62.44    | FUTR 16.06   
Model: LR_ThresDP                   | ACC 0.670±0.008 (-13.16

Data: compas_race | Model: Dummy                        : 100%|██████████| 5/5 [00:00<00:00, 17.34it/s, ACC 0.491±0.015 | BACC 0.489±0.016 | DP 0.014±0.012 | EO 0.035±0.023 | AdvG {1: 3, 0: 2}]
Data: compas_race | Model: LR                           : 100%|██████████| 5/5 [00:00<00:00,  5.80it/s, ACC 0.685±0.011 | BACC 0.680±0.011 | DP 0.310±0.028 | EO 0.337±0.036 | AdvG {1: 5}]
Data: compas_race | Model: LR_Reweight                  : 100%|██████████| 5/5 [00:03<00:00,  1.65it/s, ACC 0.677±0.012 | BACC 0.667±0.012 | DP 0.275±0.028 | EO 0.319±0.040 | AdvG {1: 5}]
Data: compas_race | Model: LR_ReductionDP               : 100%|██████████| 5/5 [00:08<00:00,  1.63s/it, ACC 0.643±0.035 | BACC 0.630±0.036 | DP 0.126±0.052 | EO 0.168±0.049 | AdvG {1: 5}]
Data: compas_race | Model: LR_ReductionEO               : 100%|██████████| 5/5 [00:07<00:00,  1.50s/it, ACC 0.652±0.039 | BACC 0.642±0.039 | DP 0.132±0.063 | EO 0.158±0.073 | AdvG {1: 5}]
Data: compas_race | Model: LR_ThresDP                 

========== Run Time on Dataset compas_race: 2m18s ==========
Results on Data: compas_race
Model: Dummy                        | ACC 0.491±0.015            | BACC 0.489±0.016            | DP 0.014±0.012            | EO 0.035±0.023           
Model: LR                           | ACC 0.685±0.011            | BACC 0.680±0.011            | DP 0.310±0.028            | EO 0.337±0.036           
Model: LR_Reweight                  | ACC 0.677±0.012 (-4.04%)   | BACC 0.667±0.012 (-7.23%)   | DP 0.275±0.028 (-11.27%)  | EO 0.319±0.040 (-5.33%)   | FURG 2.66     | FUTR 1.47    
Model: LR_ReductionDP               | ACC 0.643±0.035 (-21.86%)  | BACC 0.630±0.036 (-26.60%)  | DP 0.126±0.052 (-59.22%)  | EO 0.168±0.049 (-50.20%)  | FURG 30.48    | FUTR 2.26    
Model: LR_ReductionEO               | ACC 0.652±0.039 (-17.21%)  | BACC 0.642±0.039 (-19.99%)  | DP 0.132±0.063 (-57.36%)  | EO 0.158±0.073 (-53.17%)  | FURG 36.66    | FUTR 2.97    
Model: LR_ThresDP                   | ACC 0.643±0.010 (-21.

...

========== Run Time on Dataset lsa_unfair_gender_race_race: 17m42s ==========
Results on Data: lsa_unfair_gender_race_race
Model: Dummy                        | ACC 0.679±0.004            | BACC 0.498±0.004            | DP 0.004±0.003            | EO 0.023±0.011           
Model: LR                           | ACC 0.835±0.002            | BACC 0.636±0.003            | DP 0.100±0.003            | EO 0.248±0.012           
Model: LR_Reweight                  | ACC 0.806±0.001 (-18.69%)  | BACC 0.513±0.001 (-89.06%)  | DP 0.007±0.000 (-93.35%)  | EO 0.026±0.003 (-89.39%)  | FURG 37.49    | FUTR 1.70    
Model: LR_ReductionDP               | ACC 0.832±0.004 (-1.68%)   | BACC 0.623±0.013 (-9.62%)   | DP 0.044±0.050 (-56.27%)  | EO 0.138±0.101 (-44.33%)  | FURG 44.65    | FUTR 8.90    
Model: LR_ReductionEO               | ACC 0.829±0.007 (-3.85%)   | BACC 0.611±0.024 (-18.22%)  | DP 0.048±0.047 (-51.89%)  | EO 0.174±0.069 (-29.71%)  | FURG 29.77    | FUTR 3.70    
Model: LR_ThresDP         

Data: bank_age | Model: Dummy                        : 100%|██████████| 5/5 [00:00<00:00,  6.62it/s, ACC 0.777±0.003 | BACC 0.499±0.006 | DP 0.016±0.016 | EO 0.052±0.037 | AdvG {0: 5}]
Data: bank_age | Model: LR                           : 100%|██████████| 5/5 [00:03<00:00,  1.62it/s, ACC 0.897±0.003 | BACC 0.678±0.005 | DP 0.103±0.033 | EO 0.144±0.106 | AdvG {1: 5}]
Data: bank_age | Model: LR_Reweight                  : 100%|██████████| 5/5 [00:26<00:00,  5.24s/it, ACC 0.875±0.001 | BACC 0.510±0.001 | DP 0.006±0.003 | EO 0.017±0.013 | AdvG {1: 3, 0: 2}]
Data: bank_age | Model: LR_ReductionDP               : 100%|██████████| 5/5 [00:45<00:00,  9.03s/it, ACC 0.896±0.002 | BACC 0.669±0.005 | DP 0.009±0.009 | EO 0.127±0.063 | AdvG {1: 3, 0: 2}]
Data: bank_age | Model: LR_ReductionEO               : 100%|██████████| 5/5 [00:43<00:00,  8.61s/it, ACC 0.897±0.003 | BACC 0.677±0.003 | DP 0.039±0.020 | EO 0.071±0.068 | AdvG {1: 5}]
Data: bank_age | Model: LR_ThresDP                   : 100%|███

========== Run Time on Dataset bank_age: 11m45s ==========
Results on Data: bank_age
Model: Dummy                        | ACC 0.777±0.003            | BACC 0.499±0.006            | DP 0.016±0.016            | EO 0.052±0.037           
Model: LR                           | ACC 0.897±0.003            | BACC 0.678±0.005            | DP 0.103±0.033            | EO 0.144±0.106           
Model: LR_Reweight                  | ACC 0.875±0.001 (-17.99%)  | BACC 0.510±0.001 (-93.96%)  | DP 0.006±0.003 (-94.46%)  | EO 0.017±0.013 (-87.90%)  | FURG 35.21    | FUTR 1.63    
Model: LR_ReductionDP               | ACC 0.896±0.002 (-0.74%)   | BACC 0.669±0.005 (-4.98%)   | DP 0.009±0.009 (-91.20%)  | EO 0.127±0.063 (-11.42%)  | FURG 48.45    | FUTR 17.93   
Model: LR_ReductionEO               | ACC 0.897±0.003 (+0.38%)   | BACC 0.677±0.003 (-0.72%)   | DP 0.039±0.020 (-62.09%)  | EO 0.071±0.068 (-50.72%)  | FURG 56.24    | FUTR 56.41   
Model: LR_ThresDP                   | ACC 0.895±0.002 (-1.18%)  

Data: bank_marital=married | Model: Dummy                        : 100%|██████████| 5/5 [00:00<00:00,  6.88it/s, ACC 0.778±0.003 | BACC 0.501±0.002 | DP 0.004±0.003 | EO 0.021±0.021 | AdvG {0: 3, 1: 2}]
Data: bank_marital=married | Model: LR                           : 100%|██████████| 5/5 [00:03<00:00,  1.59it/s, ACC 0.900±0.001 | BACC 0.688±0.006 | DP 0.021±0.006 | EO 0.037±0.031 | AdvG {1: 5}]
Data: bank_marital=married | Model: LR_Reweight                  : 100%|██████████| 5/5 [00:26<00:00,  5.36s/it, ACC 0.875±0.001 | BACC 0.512±0.002 | DP 0.003±0.002 | EO 0.014±0.010 | AdvG {1: 4, 0: 1}]
Data: bank_marital=married | Model: LR_ReductionDP               : 100%|██████████| 5/5 [00:47<00:00,  9.59s/it, ACC 0.900±0.001 | BACC 0.688±0.006 | DP 0.021±0.006 | EO 0.037±0.030 | AdvG {1: 5}]
Data: bank_marital=married | Model: LR_ReductionEO               : 100%|██████████| 5/5 [00:51<00:00, 10.26s/it, ACC 0.900±0.001 | BACC 0.688±0.006 | DP 0.021±0.006 | EO 0.037±0.030 | AdvG {1: 5}]
Dat

========== Run Time on Dataset bank_marital=married: 12m49s ==========
Results on Data: bank_marital=married
Model: Dummy                        | ACC 0.778±0.003            | BACC 0.501±0.002            | DP 0.004±0.003            | EO 0.021±0.021           
Model: LR                           | ACC 0.900±0.001            | BACC 0.688±0.006            | DP 0.021±0.006            | EO 0.037±0.031           
Model: LR_Reweight                  | ACC 0.875±0.001 (-20.24%)  | BACC 0.512±0.002 (-94.08%)  | DP 0.003±0.002 (-86.34%)  | EO 0.014±0.010 (-62.83%)  | FURG 17.42    | FUTR 1.30    
Model: LR_ReductionDP               | ACC 0.900±0.001 (+0.00%)   | BACC 0.688±0.006 (+0.00%)   | DP 0.021±0.006 (-0.64%)   | EO 0.037±0.030 (+0.00%)   | FURG 0.32     | FUTR 0.32    
Model: LR_ReductionEO               | ACC 0.900±0.001 (+0.00%)   | BACC 0.688±0.006 (+0.00%)   | DP 0.021±0.006 (-0.64%)   | EO 0.037±0.030 (+0.00%)   | FURG 0.32     | FUTR 0.32    
Model: LR_ThresDP                   | AC

In [4]:
# benchmark.save_results(path='./results', prefix='Benchmark')

Saving results to ./results/Benchmark_nBaseModel(6)_nBaseline(11)_nDataset(8)_nRuns(5)_RandSeed(42)_DummyStrategy(stratified)_Time(2023-08-08-02-53-02).csv ... Done.
